# Ultimate Crowd Counter 2000

For the PYNQ Bootcamp Hackathon 2023, we present a new tool for crowd counting -- **the ultimate crowd counter**!

> Team Members: Coleman Gamble, André Rösti
> 
> Model Used: pt_BCC.xmodel
> 
> Peripherals Used: LED Bar (optional)
> 
> Goal of the Project: Indicate the number of people in a crowd

### (1) Imports

We used the standard imports, like in the trainings.

In [11]:
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from pynq_dpu import DpuOverlay
from pynq_peripherals import PmodGroveAdapter
import ipywidgets as widgets

### (2) Set Up Overlay, LED Bar, and Load Model

First, we set up the FPGA with a DPU overlay. Then we load our "Bayesian Crowd Counting" model. We also set up the LED bar.

In [5]:
overlay = DpuOverlay("dpu.bit")
adapter = PmodGroveAdapter(overlay.PMODA, G2='grove_ledbar')
ledbar = adapter.G2
overlay.load_model("pt_BCC.xmodel")
dpu = overlay.runner

### (3) Preprocessing Code

For preprocessing, we had to do the following things:
1. Open the image at the given path and convert it into an RGB numpy array.
2. Resize the image to the correct dimensions: 1000 x 800
3. Normalize the RGB values from 0 - 255 to 0 - 1.
4. Perform "Z score normalization", something this model requires that we copied from the paper.

In [6]:
def preprocess(path):
    img_pil = Image.open(path).convert('RGB')
    img_np = np.array(img_pil)
    img_h, img_w, img_c = img_np.shape
    img = img_np
    img = Image.fromarray(img.astype('uint8'), 'RGB')#  convert to pil image
    img = img.resize((1000, 800)) # resize to 1000 x 800 image
    img = 1.0 * np.array(img) / 255 # normalize to 0, 1
    img -= np.array([0.485, 0.456, 0.406]) 
    img /= np.array([0.229, 0.224, 0.225]) # z score normalization (calculated from training dataset)
    return img.astype(np.float32, order="C") # order "C" switchs rows and columns

### (4) Inference Code

We first obtain the correct input and output dimensions for this model on the DPU, and prepare some arrays to hold the inputs and outputs. Then, we run the model on the preprocessed input image.

In [42]:
def inference(input_img):
    inputTensors = dpu.get_input_tensors()
    outputTensors = dpu.get_output_tensors()
    shapeIn = tuple(inputTensors[0].dims)
    shapeOut = tuple(outputTensors[0].dims)
    outputSize = int(outputTensors[0].get_data_size() / shapeIn[0])
    output_data = [np.zeros(shapeOut, dtype=np.float32, order="C")]
    input_data = [np.zeros(shapeIn, dtype=np.float32, order="C")]
    input_data[0] = input_img
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    return output_data

### (5) Post-Processing Code

Post processing is pretty simple: The total estimate of number of people is simply the sum of the output array.

In [8]:
def postprocessing(output_data):
    final_count = np.sum(output_data[0]) # sum ouputs for all "key points"
    return final_count

### (5) LED Bar Code

We set the LED level to indicate the amount of people on a scale form 0 (only one LED) - 6000 (all 10 LEDs light up).

In [9]:
def setLedLevel(final_count):
    MAX_NUM_PEOPLE = 6000
    brightness = 3  # brightness of led bars
    green_to_red = 1 # 
    level = (10 * final_count) // MAX_NUM_PEOPLE # calculate level to set on ledbar
    level = int(level) # requires integer
    if(level <= 0):
        level = 1
    elif(level > 10):
        level = 10
    ledbar.set_level(int(level), brightness, green_to_red)

### (6) Widgets / File Upload / Processing Code

To make everything look a little nicer, we call these functions whenever a button is clicked.

In [69]:
fileSelector = None
button = None
label = None

def createFileSelector():
    return widgets.FileUpload(
            accept='',
            multiple=False
    )

def upload():
    filename = "--tmp-" + next(iter(fileSelector.value.keys()))
    file = next(iter(fileSelector.value.values()))
    with open(filename, "wb") as fp:
        fp.write(file["content"])
    return filename
        
def main(x):
    try:
        filename = upload()
        img = preprocess(filename)
        output_data = inference(img)
        final_count = postprocessing(output_data)
        label.value = "<p style='float:left; margin-right:20px;'><img src='{0}' width='350' /></p><p>There are approximately <strong>{1:.0f}</strong> people in this photo.</p><hr style='clear:both' />".format(filename, final_count)
        try:
            setLedLevel(final_count)
        except:
            pass  # Ignore LED errors
    except BaseException as e:
        label.value = "<p>Something went wrong: <code>{}</code></p>".format(e)

fileSelector = createFileSelector()

button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='',
    tooltip='Click me',
    icon='check'
)

header = widgets.HTML(value="<hr/><hr/><p style='text-align:center'><img src='./img/crowd_estimator_logo.png' /></p>")
label = widgets.HTML(value="<p>Welcome to Ultimate Crowd Estimator 2000</p>")

button.on_click(main)

In [70]:
display(header)
display(label)
display(fileSelector)
display(button)

HTML(value="<hr/><hr/><p style='text-align:center'><img src='./crowd_estimator_logo.png' /></p>")

HTML(value='<p>Welcome to Ultimate Crowd Estimator 2000</p>')

FileUpload(value={}, description='Upload')

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

<hr />
<hr />

In [ ]:
ledbar.clear()
del dpu
del overlay